In [1]:
import os
import pandas as pd
import numpy
import pickle
import pefile
from sklearn import metrics , svm
import sklearn.ensemble as ek
from sklearn.model_selection import cross_validate 
from sklearn.model_selection import train_test_split
from sklearn import tree, linear_model
from sklearn.feature_selection import SelectFromModel
#from sklearn.externals import joblib
import joblib
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from sklearn import svm
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import IsolationForest 
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm.libsvm import decision_function

C:\Users\Sarthak\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.svm.libsvm module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.svm. Anything that cannot be imported from sklearn.svm is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
dataset = pd.read_csv(r'C:\Users\Sarthak\Downloads\malware_classification\data_update_new.csv', low_memory = False)

In [4]:
dataset

,Name,md5,Machine,SizeOfOptionalHeader,Characteristics,MajorLinkerVersion,MinorLinkerVersion,SizeOfCode,SizeOfInitializedData,SizeOfUninitializedData,...,ResourcesNb,ResourcesMeanEntropy,ResourcesMinEntropy,ResourcesMaxEntropy,ResourcesMeanSize,ResourcesMinSize,ResourcesMaxSize,LoadConfigurationSize,VersionInformationSize,legitimate
0,memtest.exe,631ea355665f28d4707448e442fbf5b8,332,224,258,9,0,361984,115712,0,...,4,3.262823,2.568844,3.537939,8797.000000,216,18032,0,16,1
1,ose.exe,9d10f99a6712e28f8acd5641e3a7ea6b,332,224,3330,9,0,130560,19968,0,...,2,4.250461,3.420744,5.080177,837.000000,518,1156,72,18,1
2,setup.exe,4d92f518527353c0db88a70fddcfd390,332,224,3330,9,0,517120,621568,0,...,11,4.426324,2.846449,5.271813,31102.272730,104,270376,72,18,1
3,DW20.EXE,a41e524f8d45f0074fd07805ff0c9b12,332,224,258,9,0,585728,369152,0,...,10,4.364291,2.669314,6.400720,1457.000000,90,4264,72,18,1
4,dwtrig20.exe,c87e561258f2f8650cef999bf643a731,332,224,258,9,0,294912,247296,0,...,2,4.306100,3.421598,5.190603,1074.500000,849,1300,72,18,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138072,Photoshop.exe,NaN,34404,240,34,14,16,176395264,57143296,0,...,820,3.523624,2.134722,7.943196,13158.647560,76,270376,256,0,1
138073,Illustrator.exe,NaN,34404,240,34,14,16,20233216,12848640,0,...,602,4.539870,1.829907,7.882860,4057.400332,28,67624,256,0,1
138074,drw.exe,NaN,332,224,33167,2,25,66560,481280,0,...,22,4.442812,2.560312,5.948442,21439.090910,16,270376,0,0,1
138075,Acrobat_DC_Web_WWMUI.exe,NaN,332,224,33166,2,25,1717248,620544,0,...,84,2.949640,0.496371,5.291240,5153.440476,2,270376,0,0,1


In [6]:
dataset.shape

(138047, 57)

In [7]:
dataset.groupby(dataset["legitimate"]).size()

legitimate
0    96724
1    41323
dtype: int64

In [3]:
dataset = dataset[:82646]
dataset.groupby(dataset["legitimate"]).size()

legitimate
0    41323
1    41323
dtype: int64

In [4]:
dataset

,Name,md5,Machine,SizeOfOptionalHeader,Characteristics,MajorLinkerVersion,MinorLinkerVersion,SizeOfCode,SizeOfInitializedData,SizeOfUninitializedData,...,ResourcesNb,ResourcesMeanEntropy,ResourcesMinEntropy,ResourcesMaxEntropy,ResourcesMeanSize,ResourcesMinSize,ResourcesMaxSize,LoadConfigurationSize,VersionInformationSize,legitimate
0,memtest.exe,631ea355665f28d4707448e442fbf5b8,332,224,258,9,0,361984,115712,0,...,4,3.262823,2.568844,3.537939,8797.000000,216,18032,0,16,1
1,ose.exe,9d10f99a6712e28f8acd5641e3a7ea6b,332,224,3330,9,0,130560,19968,0,...,2,4.250461,3.420744,5.080177,837.000000,518,1156,72,18,1
2,setup.exe,4d92f518527353c0db88a70fddcfd390,332,224,3330,9,0,517120,621568,0,...,11,4.426324,2.846449,5.271813,31102.272727,104,270376,72,18,1
3,DW20.EXE,a41e524f8d45f0074fd07805ff0c9b12,332,224,258,9,0,585728,369152,0,...,10,4.364291,2.669314,6.400720,1457.000000,90,4264,72,18,1
4,dwtrig20.exe,c87e561258f2f8650cef999bf643a731,332,224,258,9,0,294912,247296,0,...,2,4.306100,3.421598,5.190603,1074.500000,849,1300,72,18,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82641,VirusShare_268135cd4e66220612f8ee88e56315ce,268135cd4e66220612f8ee88e56315ce,332,224,258,8,0,138752,1536,0,...,1,3.164791,3.164791,3.164791,580.000000,580,580,0,15,0
82642,VirusShare_b4f1a7ab2283225bd84be650e054141f,b4f1a7ab2283225bd84be650e054141f,332,224,258,10,0,119808,385024,0,...,6,3.776760,2.458492,5.317552,2740.166667,48,9640,72,15,0
82643,VirusShare_637da90920b6dbe8f67e3bf0357f5984,637da90920b6dbe8f67e3bf0357f5984,332,224,258,11,0,113664,682496,0,...,7,4.453368,2.623078,7.967002,80992.857143,62,534528,0,0,0
82644,VirusShare_b3c719c73f1dd7470f2cbc919ef88063,b3c719c73f1dd7470f2cbc919ef88063,332,224,258,10,0,117760,392192,0,...,5,3.425849,2.458492,4.035538,3146.400000,48,9640,72,15,0


In [5]:
#sperating columns

x = dataset.drop(["Name","md5","legitimate"],axis =1).values

y = dataset["legitimate"].values

In [11]:
# ExtraTreesClassifier

# ExtraTreesClassifier fits a number of randomized decision trees (a.k.a. extra-trees) on various sub-samples of the dataset and use averaging to improve the predictive accuracy and control over-fitting

In [6]:
extratree = ExtraTreesClassifier().fit(x,y)
model = SelectFromModel(extratree,prefit = True)
X_new = model.transform(x)


In [7]:
nbfeatures = X_new.shape[1]

In [8]:
nbfeatures

13

In [15]:
# Cross Validation

# Cross validation is applied to divide the dataset into random train and test subsets. test_size = 0.2 represent the proportion of the dataset to include in the test split

In [16]:
X_new.shape

(82646, 13)

In [9]:
X_train, X_test, Y_train, Y_test =train_test_split(X_new, y ,test_size=0.2)

In [10]:
X_train.shape

(66116, 13)

In [11]:
features = []
index = numpy.argsort(extratree.feature_importances_)[::-1][:nbfeatures]

In [12]:
# The features identified by ExtraTreesClassifier

In [13]:
for f in range(nbfeatures):
    print("%d. feature %s (%f)" % (f + 1, dataset.columns[2+index[f]], extratree.feature_importances_[index[f]]))
    features.append(dataset.columns[2+f])

1. feature DllCharacteristics (0.168828)
2. feature Characteristics (0.090161)
3. feature VersionInformationSize (0.088919)
4. feature Machine (0.084998)
5. feature SectionsMaxEntropy (0.062493)
6. feature Subsystem (0.056740)
7. feature SizeOfOptionalHeader (0.055011)
8. feature ImageBase (0.047511)
9. feature MajorSubsystemVersion (0.044777)
10. feature ResourcesMaxEntropy (0.036025)
11. feature ResourcesMinEntropy (0.029012)
12. feature SizeOfStackReserve (0.019863)
13. feature SectionsMeanEntropy (0.019503)


In [14]:
#BUILDING MODELS

In [15]:
model = { "DecisionTree":tree.DecisionTreeClassifier(max_depth=10),
         "RandomForest":ek.RandomForestClassifier(n_estimators=50,criterion = "entropy"),
         "Adaboost":ek.AdaBoostClassifier(n_estimators=50),
         "GradientBoosting":ek.GradientBoostingClassifier(n_estimators=50),
         "GNB":GaussianNB(),
         "LinearRegression":LinearRegression() ,
         #"svm_rbf":  svm.SVC(kernel = 'rbf', gamma =0.1,C = 1.0),
         "svm_linear":  svm.SVC(kernel = 'linear')
    
}

In [ ]:
#NOW CHECK EACH ACCURACY

results = {}
for algo in model:
    clf = model[algo]
    clf.fit(X_train,Y_train)
    score = clf.score(X_test,Y_test)
    print ("%s : %s " %(algo, score))
    results[algo] = score

DecisionTree : 0.9895946763460375 
RandomForest : 0.9930429522081065 
Adaboost : 0.9842105263157894 
GradientBoosting : 0.9871143375680581 
GNB : 0.5015728977616455 
LinearRegression : 0.589863488572183 


In [25]:
winner = max(results, key=results.get)

In [26]:
#calculating false positive
clf = model[winner]
res = clf.predict(X_new)
mt = confusion_matrix(y, res)
print("False positive rate : %f %%" % ((mt[0][1] / float(sum(mt[0])))*100))
print('False negative rate : %f %%' % ( (mt[1][0] / float(sum(mt[1]))*100)))

False positive rate : 0.183917 %
False negative rate : 0.106478 %


In [27]:
#SAVING MODEL
joblib.dump(model[winner],r'C:\Users\Sarthak\Downloads\malware_classification/classifier.pkl')

['C:\\Users\\Sarthak\\Downloads\\malware_classification/classifier.pkl']

In [28]:
joblib.dump(features,r'C:\Users\Sarthak\Downloads\malware_classification/features.pkl')

['C:\\Users\\Sarthak\\Downloads\\malware_classification/features.pkl']

In [44]:
open(r'C:\Users\Sarthak\Downloads\malware_classification/features.pkl', 'wb').write(pickle.dumps(features))

315

In [5]:
# Load classifier
clf = joblib.load(r'C:\Users\Sarthak\Downloads\malware_classification/classifier.pkl')
#load features
# features = pickle.loads(open(os.path.join(r'C:\Users\Sarthak\Downloads\malware_classification/features.pkl'),'rb').read())

features = joblib.load(r'C:\Users\Sarthak\Downloads\malware_classification/features.pkl')

In [30]:
# Testing with unseen file
# Given any unseen test file, it's required to extract the characteristics of the given file.

# In order to test the model on an unseen file, it's required to extract the characteristics of the given file. Python's pefile.PE library is used to construct and build the feature vector and a ML model is used to predict the class for the given file based on the already trained model.

In [40]:
# %load malware_test.py
"""
this file extracts the required information of a given file using the library PE 

"""

import pefile
import os
import array
import math
import pickle
import joblib
import sys
import argparse
import joblib


def get_entropy(data):
    if len(data) == 0:
        return 0.0
    occurences = array.array('L', [0]*256)
    for x in data:
        occurences[x if isinstance(x, int) else ord(x)] += 1

    entropy = 0
    for x in occurences:
        if x:
            p_x = float(x) / len(data)
            entropy -= p_x*math.log(p_x, 2)

    return entropy


def get_resources(pe):
    """Extract resources :
    [entropy, size]"""
    resources = []
    if hasattr(pe, 'DIRECTORY_ENTRY_RESOURCE'):
        try:
            for resource_type in pe.DIRECTORY_ENTRY_RESOURCE.entries:
                if hasattr(resource_type, 'directory'):
                    for resource_id in resource_type.directory.entries:
                        if hasattr(resource_id, 'directory'):
                            for resource_lang in resource_id.directory.entries:
                                data = pe.get_data(resource_lang.data.struct.OffsetToData, resource_lang.data.struct.Size)
                                size = resource_lang.data.struct.Size
                                entropy =get_entropy(data)

                                resources.append([entropy, size])
    
        except Exception as e:
            return resources
    return resources
    

def get_version_info(pe):
    """Return version infos"""
    res = {}
    for fileinfo in pe.FileInfo:
        if fileinfo.Key == 'StringFileInfo':
            for st in fileinfo.StringTable:
                for entry in st.entries.items():
                    res[entry[0]] = entry[1]
        if fileinfo.Key == 'VarFileInfo':
            for var in fileinfo.Var:
                res[var.entry.items()[0][0]] = var.entry.items()[0][1]
    if hasattr(pe, 'VS_FIXEDFILEINFO'):
          res['flags'] = pe.VS_FIXEDFILEINFO.FileFlags
          res['os'] = pe.VS_FIXEDFILEINFO.FileOS
          res['type'] = pe.VS_FIXEDFILEINFO.FileType
          res['file_version'] = pe.VS_FIXEDFILEINFO.FileVersionLS
          res['product_version'] = pe.VS_FIXEDFILEINFO.ProductVersionLS
          res['signature'] = pe.VS_FIXEDFILEINFO.Signature
          res['struct_version'] = pe.VS_FIXEDFILEINFO.StrucVersion
    return res

#extract the info for a given file
def extract_infos(fpath):
    res = {}
    pe = pefile.PE(fpath)
    res['Machine'] = pe.FILE_HEADER.Machine
    res['SizeOfOptionalHeader'] = pe.FILE_HEADER.SizeOfOptionalHeader
    res['Characteristics'] = pe.FILE_HEADER.Characteristics
    res['MajorLinkerVersion'] = pe.OPTIONAL_HEADER.MajorLinkerVersion
    res['MinorLinkerVersion'] = pe.OPTIONAL_HEADER.MinorLinkerVersion
    res['SizeOfCode'] = pe.OPTIONAL_HEADER.SizeOfCode
    res['SizeOfInitializedData'] = pe.OPTIONAL_HEADER.SizeOfInitializedData
    res['SizeOfUninitializedData'] = pe.OPTIONAL_HEADER.SizeOfUninitializedData
    res['AddressOfEntryPoint'] = pe.OPTIONAL_HEADER.AddressOfEntryPoint
    res['BaseOfCode'] = pe.OPTIONAL_HEADER.BaseOfCode
    try:
        res['BaseOfData'] = pe.OPTIONAL_HEADER.BaseOfData
    except AttributeError:
        res['BaseOfData'] = 0
    res['ImageBase'] = pe.OPTIONAL_HEADER.ImageBase
    res['SectionAlignment'] = pe.OPTIONAL_HEADER.SectionAlignment
    res['FileAlignment'] = pe.OPTIONAL_HEADER.FileAlignment
    res['MajorOperatingSystemVersion'] = pe.OPTIONAL_HEADER.MajorOperatingSystemVersion
    res['MinorOperatingSystemVersion'] = pe.OPTIONAL_HEADER.MinorOperatingSystemVersion
    res['MajorImageVersion'] = pe.OPTIONAL_HEADER.MajorImageVersion
    res['MinorImageVersion'] = pe.OPTIONAL_HEADER.MinorImageVersion
    res['MajorSubsystemVersion'] = pe.OPTIONAL_HEADER.MajorSubsystemVersion
    res['MinorSubsystemVersion'] = pe.OPTIONAL_HEADER.MinorSubsystemVersion
    res['SizeOfImage'] = pe.OPTIONAL_HEADER.SizeOfImage
    res['SizeOfHeaders'] = pe.OPTIONAL_HEADER.SizeOfHeaders
    res['CheckSum'] = pe.OPTIONAL_HEADER.CheckSum
    res['Subsystem'] = pe.OPTIONAL_HEADER.Subsystem
    res['DllCharacteristics'] = pe.OPTIONAL_HEADER.DllCharacteristics
    res['SizeOfStackReserve'] = pe.OPTIONAL_HEADER.SizeOfStackReserve
    res['SizeOfStackCommit'] = pe.OPTIONAL_HEADER.SizeOfStackCommit
    res['SizeOfHeapReserve'] = pe.OPTIONAL_HEADER.SizeOfHeapReserve
    res['SizeOfHeapCommit'] = pe.OPTIONAL_HEADER.SizeOfHeapCommit
    res['LoaderFlags'] = pe.OPTIONAL_HEADER.LoaderFlags
    res['NumberOfRvaAndSizes'] = pe.OPTIONAL_HEADER.NumberOfRvaAndSizes

    # Sections
    res['SectionsNb'] = len(pe.sections)

    entropy = list(map(lambda x:x.get_entropy(), pe.sections))
    res['SectionsMeanEntropy'] = sum(entropy)/float(len(entropy))
    res['SectionsMinEntropy'] = min(entropy)
    res['SectionsMaxEntropy'] = max(entropy)
    raw_sizes = list(map(lambda x:x.SizeOfRawData, pe.sections))
    res['SectionsMeanRawsize'] = sum(raw_sizes)/float(len(raw_sizes))
    res['SectionsMinRawsize'] = min(raw_sizes)
    res['SectionsMaxRawsize'] = max(raw_sizes)
    virtual_sizes = list(map(lambda x:x.Misc_VirtualSize, pe.sections))
    res['SectionsMeanVirtualsize'] = sum(virtual_sizes)/float(len(virtual_sizes))
    res['SectionsMinVirtualsize'] = min(virtual_sizes)
    res['SectionMaxVirtualsize'] = max(virtual_sizes)

    #Imports
    try:
        res['ImportsNbDLL'] = len(pe.DIRECTORY_ENTRY_IMPORT)
        imports = sum([x.imports for x in pe.DIRECTORY_ENTRY_IMPORT], [])
        
        res['ImportsNb'] = len(imports)
        res['ImportsNbOrdinal'] = len(list(filter(lambda x:x.name is None, imports)))
    except AttributeError:
        res['ImportsNbDLL'] = 0
        res['ImportsNb'] = 0
        res['ImportsNbOrdinal'] = 0

    #Exports
    try:
        res['ExportNb'] = len(pe.DIRECTORY_ENTRY_EXPORT.symbols)
    except AttributeError:
        # No export
        res['ExportNb'] = 0
    #Resources
    resources= get_resources(pe)
    res['ResourcesNb'] = len(resources)
    if len(resources)> 0:
        entropy = list(map(lambda x:x[0], resources))
        res['ResourcesMeanEntropy'] = sum(entropy)/float(len(entropy))
        res['ResourcesMinEntropy'] = min(entropy)
        res['ResourcesMaxEntropy'] = max(entropy)
        sizes = list(map(lambda x:x[1], resources))
        res['ResourcesMeanSize'] = sum(sizes)/float(len(sizes))
        res['ResourcesMinSize'] = min(sizes)
        res['ResourcesMaxSize'] = max(sizes)
    else:
        res['ResourcesNb'] = 0
        res['ResourcesMeanEntropy'] = 0
        res['ResourcesMinEntropy'] = 0
        res['ResourcesMaxEntropy'] = 0
        res['ResourcesMeanSize'] = 0
        res['ResourcesMinSize'] = 0
        res['ResourcesMaxSize'] = 0

    # Load configuration size
    try:
        res['LoadConfigurationSize'] = pe.DIRECTORY_ENTRY_LOAD_CONFIG.struct.Size
    except AttributeError:
        res['LoadConfigurationSize'] = 0


    # Version configuration size
    try:
        version_infos = get_version_info(pe)
        res['VersionInformationSize'] = len(version_infos.keys())
    except AttributeError:
        res['VersionInformationSize'] = 0
    return res


if __name__ == '__main__':
    clf = joblib.load(r'C:\Users\Sarthak\Downloads\malware_classification/classifier.pkl')
    features = joblib.load(r'C:\Users\Sarthak\Downloads\malware_classification/features.pkl')
    data = extract_infos(r'C:\Users\Sarthak\Downloads\wetransfer-5062c7\burpsuite_community_windows-x64_v2020_8_1.exe')
#     data = dataset.iloc[82641]
    print(data)
#     pe_features = list(map(lambda x:data[x], features))Skype-8.61.0.87.exe
#     res= clf.predict([pe_features])[0]   
#     print(res)
#     print ('The file %s is %s' % (os.path.basename(sys.argv[1]),[ 'malicious','legitimate'][res]))

{'Machine': 34404, 'SizeOfOptionalHeader': 240, 'Characteristics': 34, 'MajorLinkerVersion': 9, 'MinorLinkerVersion': 0, 'SizeOfCode': 296960, 'SizeOfInitializedData': 359936, 'SizeOfUninitializedData': 0, 'AddressOfEntryPoint': 208392, 'BaseOfCode': 4096, 'BaseOfData': 0, 'ImageBase': 5368709120, 'SectionAlignment': 4096, 'FileAlignment': 512, 'MajorOperatingSystemVersion': 5, 'MinorOperatingSystemVersion': 2, 'MajorImageVersion': 0, 'MinorImageVersion': 0, 'MajorSubsystemVersion': 5, 'MinorSubsystemVersion': 2, 'SizeOfImage': 667648, 'SizeOfHeaders': 1024, 'CheckSum': 0, 'Subsystem': 2, 'DllCharacteristics': 33088, 'SizeOfStackReserve': 1048576, 'SizeOfStackCommit': 4096, 'SizeOfHeapReserve': 1048576, 'SizeOfHeapCommit': 4096, 'LoaderFlags': 0, 'NumberOfRvaAndSizes': 16, 'SectionsNb': 6, 'SectionsMeanEntropy': 4.590615240394957, 'SectionsMinEntropy': 2.6815948831197307, 'SectionsMaxEntropy': 6.4240788869902365, 'SectionsMeanRawsize': 97536.0, 'SectionsMinRawsize': 2560, 'SectionsMaxR

In [5]:
len(data)

54

In [41]:
dataset = dataset.append({"Name":"burpsuite_community_windows-x64_v2020_8_1.exe",'Machine': 34404, 'SizeOfOptionalHeader': 240, 'Characteristics': 34, 'MajorLinkerVersion': 9, 'MinorLinkerVersion': 0, 'SizeOfCode': 296960, 'SizeOfInitializedData': 359936, 'SizeOfUninitializedData': 0, 'AddressOfEntryPoint': 208392, 'BaseOfCode': 4096, 'BaseOfData': 0, 'ImageBase': 5368709120, 'SectionAlignment': 4096, 'FileAlignment': 512, 'MajorOperatingSystemVersion': 5, 'MinorOperatingSystemVersion': 2, 'MajorImageVersion': 0, 'MinorImageVersion': 0, 'MajorSubsystemVersion': 5, 'MinorSubsystemVersion': 2, 'SizeOfImage': 667648, 'SizeOfHeaders': 1024, 'CheckSum': 0, 'Subsystem': 2, 'DllCharacteristics': 33088, 'SizeOfStackReserve': 1048576, 'SizeOfStackCommit': 4096, 'SizeOfHeapReserve': 1048576, 'SizeOfHeapCommit': 4096, 'LoaderFlags': 0, 'NumberOfRvaAndSizes': 16, 'SectionsNb': 6, 'SectionsMeanEntropy': 4.590615240394957, 'SectionsMinEntropy': 2.6815948831197307, 'SectionsMaxEntropy': 6.4240788869902365, 'SectionsMeanRawsize': 97536.0, 'SectionsMinRawsize': 2560, 'SectionsMaxRawsize': 296960, 'SectionsMeanVirtualsize': 109234.16666666667, 'SectionsMinVirtualsize': 2380, 'SectionMaxVirtualsize': 296586, 'ImportsNbDLL': 5, 'ImportsNb': 253, 'ImportsNbOrdinal': 0, 'ExportNb': 84, 'ResourcesNb': 16, 'ResourcesMeanEntropy': 2.87007465153923, 'ResourcesMinEntropy': 1.8542572100359584, 'ResourcesMaxEntropy': 5.195522870887146, 'ResourcesMeanSize': 8466.3125, 'ResourcesMinSize': 146, 'ResourcesMaxSize': 67624, 'LoadConfigurationSize': 0, 'VersionInformationSize': 0,"legitimate":1},ignore_index=True)

In [42]:
dataset

,Name,md5,Machine,SizeOfOptionalHeader,Characteristics,MajorLinkerVersion,MinorLinkerVersion,SizeOfCode,SizeOfInitializedData,SizeOfUninitializedData,...,ResourcesNb,ResourcesMeanEntropy,ResourcesMinEntropy,ResourcesMaxEntropy,ResourcesMeanSize,ResourcesMinSize,ResourcesMaxSize,LoadConfigurationSize,VersionInformationSize,legitimate
0,memtest.exe,631ea355665f28d4707448e442fbf5b8,332,224,258,9,0,361984,115712,0,...,4,3.262823,2.568844,3.537939,8797.000000,216,18032,0,16,1
1,ose.exe,9d10f99a6712e28f8acd5641e3a7ea6b,332,224,3330,9,0,130560,19968,0,...,2,4.250461,3.420744,5.080177,837.000000,518,1156,72,18,1
2,setup.exe,4d92f518527353c0db88a70fddcfd390,332,224,3330,9,0,517120,621568,0,...,11,4.426324,2.846449,5.271813,31102.272730,104,270376,72,18,1
3,DW20.EXE,a41e524f8d45f0074fd07805ff0c9b12,332,224,258,9,0,585728,369152,0,...,10,4.364291,2.669314,6.400720,1457.000000,90,4264,72,18,1
4,dwtrig20.exe,c87e561258f2f8650cef999bf643a731,332,224,258,9,0,294912,247296,0,...,2,4.306100,3.421598,5.190603,1074.500000,849,1300,72,18,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138082,SearchUI.exe,NaN,34404,240,34,14,15,6347264,4950528,0,...,1,3.501474,3.501474,3.501474,940.000000,940,940,264,0,1
138083,tdm64-gcc-9.2.0.exe,NaN,332,224,271,6,0,20480,16384,299008,...,8,4.958342,1.545472,7.456178,1644.250000,34,9640,0,0,1
138084,UwAmp.exe,NaN,332,224,271,6,0,23552,119808,1024,...,14,4.102813,2.521832,6.266115,1133.357143,104,4264,0,0,1
138085,xampp-windows-x64-7.2.33-0-VC15-installer.exe,NaN,34404,240,558,2,25,2163200,3092480,16384,...,169,2.268854,1.141507,7.969157,750.674556,20,30732,0,0,1


In [43]:
dataset.to_csv(r'C:\Users\Sarthak\Downloads\data_update_new.csv')

In [31]:
dataset.isnull().sum()

Name                            0
md5                            37
Machine                         0
SizeOfOptionalHeader            0
Characteristics                 0
MajorLinkerVersion              0
MinorLinkerVersion              0
SizeOfCode                      0
SizeOfInitializedData           0
SizeOfUninitializedData         0
AddressOfEntryPoint             0
BaseOfCode                      0
BaseOfData                      0
ImageBase                       0
SectionAlignment                0
FileAlignment                   0
MajorOperatingSystemVersion     0
MinorOperatingSystemVersion     0
MajorImageVersion               0
MinorImageVersion               0
MajorSubsystemVersion           0
MinorSubsystemVersion           0
SizeOfImage                     0
SizeOfHeaders                   0
CheckSum                        0
Subsystem                       0
DllCharacteristics              0
SizeOfStackReserve              0
SizeOfStackCommit               0
SizeOfHeapRese